# CytoTable from the cloud (using cloud-based data sources)

![Image showing feature data from the cloud being read by CytoTable and exported to a CytoTable file afterwards.](../_static/features_to_cytotable_cloud.png)

__Figure 1.__ _CytoTable is capable of reading data from cloud-based locations such as AWS S3._

This notebook includes a quick demonstration of CytoTable with cloud-based data sources.
For a more general overview of using CytoTable and the concepts behind the work please see: [CytoTable mise en place (general overview)](https://cytomining.github.io/CytoTable/examples/cytotable_mise_en_place_general_overview.html)

In [1]:
import pathlib
from collections import Counter

import cytotable
import pandas as pd
import pyarrow.parquet as pq
from cloudpathlib import CloudPath, S3Client

## Using CytoTable with cloud-based CSV's

In [2]:
# setup variables for use in this section of the notebook
source_path = (
    "s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/"
    "analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/"
)
dest_path = "./cloud_example.parquet"

In [3]:
# remove the dest_path if it's present
if pathlib.Path(dest_path).is_file():
    pathlib.Path(dest_path).unlink()

In [4]:
# setup a source cloudpath using unsigned (anonymous) requests to AWS S3
# to access publicly-available data using CytoTable
source_cloud_path = S3Client(no_sign_request=True).CloudPath(source_path)
print(source_cloud_path)
# show the files within the path
list(source_cloud_path.glob("*"))

s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/


[S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/outlines'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/Cells.csv'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/CellsIncludingEdges.csv'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/Cytoplasm.csv'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/Experiment.csv'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2/BR00146052/analysis/BR00146052-A01-1/Image.csv'),
 S3Path('s3://cellpainting-gallery/cpg0037-oasis/broad/workspace/analysis/2025_01_12_U2OS_Batch2

In [5]:
%%time

# run cytotable convert
result = cytotable.convert(
    source_path=source_path,
    source_datatype="csv",
    # Set the chunk size for paginated processing.
    # Smaller values use fewer resources but take longer;
    # larger values use more resources and process faster.
    chunk_size=30000,
    # specify the destination path
    dest_path=dest_path,
    # specify a destination data format type
    dest_datatype="parquet",
    # specify a preset which enables quick use of common input file formats
    preset="cellprofiler_csv",
    # use unsigned (anonymous) requests to AWS S3
    no_sign_request=True,
)
print(pathlib.Path(result).name)

cloud_example.parquet
CPU times: user 561 ms, sys: 411 ms, total: 972 ms
Wall time: 34.6 s


In [6]:
# show the table head using pandas
pq.read_table(source=result).to_pandas().head()

,Metadata_ImageNumber,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ObjectNumber,Image_FileName_CellOutlines,Image_FileName_IllumAGP,Image_FileName_IllumBrightfield,Image_FileName_IllumDNA,Image_FileName_IllumER,Image_FileName_IllumMito,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,1,1,1,1,A01_s1--cell_outlines.png,BR00146052_IllumAGP.npy,BR00146052_IllumBrightfield.npy,BR00146052_IllumDNA.npy,BR00146052_IllumER.npy,BR00146052_IllumMito.npy,...,87.268467,96.580786,80.795675,81.534795,83.061909,82.135369,82.214610,83.472095,83.587552,85.043910
1,1,2,2,2,A01_s1--cell_outlines.png,BR00146052_IllumAGP.npy,BR00146052_IllumBrightfield.npy,BR00146052_IllumDNA.npy,BR00146052_IllumER.npy,BR00146052_IllumMito.npy,...,82.909202,80.373067,81.970114,81.418564,83.180896,82.860898,78.857701,80.516994,83.621363,85.743050
2,1,3,3,3,A01_s1--cell_outlines.png,BR00146052_IllumAGP.npy,BR00146052_IllumBrightfield.npy,BR00146052_IllumDNA.npy,BR00146052_IllumER.npy,BR00146052_IllumMito.npy,...,26.404112,26.620592,24.788549,25.995656,24.661882,26.114983,26.546437,26.839052,26.655321,29.181173
3,1,4,4,4,A01_s1--cell_outlines.png,BR00146052_IllumAGP.npy,BR00146052_IllumBrightfield.npy,BR00146052_IllumDNA.npy,BR00146052_IllumER.npy,BR00146052_IllumMito.npy,...,67.762866,142.014839,100.859940,106.726220,118.372463,102.754337,99.659909,108.074924,111.277440,108.468769
4,1,5,5,5,A01_s1--cell_outlines.png,BR00146052_IllumAGP.npy,BR00146052_IllumBrightfield.npy,BR00146052_IllumDNA.npy,BR00146052_IllumER.npy,BR00146052_IllumMito.npy,...,2859.404170,2317.858027,2883.111739,2923.312269,2835.480838,2883.548140,2905.259210,2941.765027,2879.090419,2799.688685


In [7]:
# show metadata for the result file
pq.read_metadata(result)

  created_by: parquet-cpp-arrow version 22.0.0
  num_columns: 4413
  num_rows: 36
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1354645

In [8]:
# show schema metadata which includes CytoTable information
# note: this information will travel with the file.
pq.read_schema(result).metadata

{b'data-producer': b'https://github.com/cytomining/CytoTable',
 b'data-producer-version': b'1.1.2.post9.dev0+323d0d2'}

In [9]:
# show schema column name summaries
# note: we have all compartments in one table
# after processing with CytoTable.
print("Column name prefix counts:")
dict(Counter(w.split("_", 1)[0] for w in pq.read_schema(result).names))

Column name prefix counts:


{'Metadata': 4, 'Image': 14, 'Cytoplasm': 1480, 'Cells': 1494, 'Nuclei': 1421}

## Using CytoTable with cloud-based SQLite databases

In [10]:
# setup variables for use in this section of the notebook
source_path = (
    "s3://cellpainting-gallery/cpg0016-jump/source_4/"
    "workspace/backend/2021_08_23_Batch12/BR00126114"
)
dest_path = "./cloud_example.parquet"

In [11]:
# remove the dest_path if it's present
if pathlib.Path(dest_path).is_file():
    pathlib.Path(dest_path).unlink()

In [12]:
# setup a source cloudpath using unsigned (anonymous) requests to AWS S3
# to access publicly-available data using CytoTable
source_cloud_path = S3Client(no_sign_request=True).CloudPath(source_path)
print(source_cloud_path)
# show the files within the path
list(source_cloud_path.glob("*"))

s3://cellpainting-gallery/cpg0016-jump/source_4/workspace/backend/2021_08_23_Batch12/BR00126114


[S3Path('s3://cellpainting-gallery/cpg0016-jump/source_4/workspace/backend/2021_08_23_Batch12/BR00126114/BR00126114.csv'),
 S3Path('s3://cellpainting-gallery/cpg0016-jump/source_4/workspace/backend/2021_08_23_Batch12/BR00126114/BR00126114.sqlite')]

In [13]:
%%time

# run cytotable convert
result = cytotable.convert(
    source_path=source_path,
    source_datatype="sqlite",
    # Set the chunk size for paginated processing.
    # Smaller values use fewer resources but take longer;
    # larger values use more resources and process faster.
    chunk_size=30000,
    # specify the destination path
    dest_path=dest_path,
    # specify a destination data format type
    dest_datatype="parquet",
    # specify a preset which enables quick use of common input file formats
    # note: presets can be customized to meet the needs of 
    # specific datasets. Check out the presets.py module for more here.
    preset="cellprofiler_sqlite_cpg0016_jump",
    # use unsigned (anonymous) requests to AWS S3
    no_sign_request=True,
    # set a local cache to avoid challenges with
    # OS-specific temp disk space
    local_cache_dir=f"./sqlite_s3_cache",
)
print(pathlib.Path(result).name)

cloud_example.parquet
CPU times: user 554 ms, sys: 350 ms, total: 904 ms
Wall time: 1min 30s


In [14]:
# show the table head using pandas
pq.read_table(source=result).to_pandas().head()

,Metadata_ImageNumber,Image_Metadata_Row,Image_Metadata_Site,Metadata_ObjectNumber,Metadata_ObjectNumber_1,Metadata_ObjectNumber_2,Metadata_Plate,Metadata_Well,Image_FileName_CellOutlines,Image_FileName_IllumAGP,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,1153,6,1,1,1,1,BR00126114,F09,F09_s1--cell_outlines.png,BR00126114_IllumAGP.npy,...,61.563533,36.992171,52.941099,51.592422,52.085098,55.278972,56.142750,48.859789,54.269368,62.026921
1,764,4,8,1,1,1,BR00126114,D13,D13_s8--cell_outlines.png,BR00126114_IllumAGP.npy,...,51.783132,54.841153,52.198763,50.045983,50.456990,50.388343,50.730273,48.807363,49.868027,50.749456
2,1585,8,1,1,1,1,BR00126114,H09,H09_s1--cell_outlines.png,BR00126114_IllumAGP.npy,...,70.238347,81.267677,72.834678,74.746135,71.591718,72.127287,75.576897,77.318713,73.304561,75.411949
3,1584,8,9,1,1,1,BR00126114,H08,H08_s9--cell_outlines.png,BR00126114_IllumAGP.npy,...,78.288859,85.116072,73.204083,75.966496,75.426898,74.496759,75.148111,74.091076,75.015060,79.454298
4,1965,10,3,1,1,1,BR00126114,J03,J03_s3--cell_outlines.png,BR00126114_IllumAGP.npy,...,119.666972,115.465740,110.179857,112.553595,109.374920,109.128350,113.458943,116.964056,111.287706,116.385013


In [15]:
# show metadata for the result file
pq.read_metadata(result)

  created_by: parquet-cpp-arrow version 22.0.0
  num_columns: 5946
  num_rows: 74226
  num_row_groups: 3
  format_version: 2.6
  serialized_size: 3709538

In [16]:
# show schema metadata which includes CytoTable information
# note: this information will travel with the file.
pq.read_schema(result).metadata

{b'data-producer': b'https://github.com/cytomining/CytoTable',
 b'data-producer-version': b'1.1.2.post9.dev0+323d0d2'}

In [17]:
# show schema column name summaries
# note: we have all compartments in one table
# after processing with CytoTable.
print("Column name prefix counts:")
dict(Counter(w.split("_", 1)[0] for w in pq.read_schema(result).names))

Column name prefix counts:


{'Metadata': 6, 'Image': 21, 'Cytoplasm': 1985, 'Cells': 1998, 'Nuclei': 1936}

In [18]:
# show full schema details
pq.read_schema(result)

Metadata_ImageNumber: int64
Image_Metadata_Row: int64
Image_Metadata_Site: int64
Metadata_ObjectNumber: int64
Metadata_ObjectNumber_1: int64
Metadata_ObjectNumber_2: int64
Metadata_Plate: string
Metadata_Well: string
Image_FileName_CellOutlines: string
Image_FileName_IllumAGP: string
Image_FileName_IllumBrightfield: string
Image_FileName_IllumBrightfield_H: string
Image_FileName_IllumBrightfield_L: string
Image_FileName_IllumDNA: string
Image_FileName_IllumER: string
Image_FileName_IllumMito: string
Image_FileName_IllumRNA: string
Image_FileName_NucleiOutlines: string
Image_FileName_OrigAGP: string
Image_FileName_OrigBrightfield: string
Image_FileName_OrigBrightfield_H: string
Image_FileName_OrigBrightfield_L: string
Image_FileName_OrigDNA: string
Image_FileName_OrigER: string
Image_FileName_OrigMito: string
Image_FileName_OrigRNA: string
Image_TableNumber: int64
Cytoplasm_AreaShape_Area: int64
Cytoplasm_AreaShape_BoundingBoxArea: int64
Cytoplasm_AreaShape_BoundingBoxMaximum_X: int64
C